# Coronagraph inpainting model

## Data downloading and preprocessing

!python3 data.py

In [1]:
from utils import get_default_device
device = get_default_device()
print(device)

cuda


In [2]:
import torch
torch.cuda.empty_cache()

## Creating dataset

In [3]:
batch_size = 1

In [4]:
from torch.utils.data import random_split, DataLoader
from utils import DeviceDataLoader
from data import CoronagraphDataset

dataset = CoronagraphDataset('c3')
# 0.8 - 0.2
train_len = round(0.8*len(dataset))
val_len = len(dataset) - train_len

#random split
train_ds, val_ds = random_split(dataset, [train_len, val_len])

train_dl = DeviceDataLoader(DataLoader(train_ds, batch_size, True, num_workers = 4, pin_memory=True), device)
val_dl = DeviceDataLoader(DataLoader(val_ds, batch_size, True, num_workers = 4, pin_memory=True), device)

/usr/lib/python3/dist-packages/paramiko/transport.py:237: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [5]:
from model import UNetArchitecture, UNetArchitectureDeluxe, SmallUNet
from utils import to_device
from loss import NewInpaintingLoss, UNetLoss, OldInpaintingLoss

# model = to_device(SingleLayer('c3', InpaintingLoss([6,6, 0.05, 10, 10]), (7,5,3)), device)

# model = to_device(UNetArchitecture('c3_UNet', UNetLoss([4, 6, 0.05, 110, 120])), device)

model = to_device(SmallUNet(f'c3_Small_Unet_{0.2}_{0.8}_{0.05}_{1}_{1.2}', NewInpaintingLoss([1., 0.8, 0.05, 100, 80]), 2), device)

1. c3_Small_Unet_0.2_0.8_0.05_1_1.2_10.pt
2. c3_Small_Unet_0.2_0.8_0.05_1_1.2_5.pt
3. c3_Small_Unet_0.2_0.8_0.05_1_1.2_0.pt


In [6]:
import torch
from utils import to_device
sample_tensor = (torch.randn(1,1,1024,1024), torch.ones(1,1,1024,1024))
sample_tensor = to_device(sample_tensor, device)

In [7]:
import torch.nn.functional as F
model.spe_act = lambda x: F.sigmoid(x)

In [8]:
from torch.optim.lr_scheduler import OneCycleLR
from torch.optim import Adam, SGD, RMSprop

model.fit(
    train_dl,
    val_dl,
    epochs = 200,
    lr = 1e-5,
    batch_size = batch_size,
    weight_decay = 0,
    grad_clip = 1e-4,
    opt_func = Adam,
    lr_sched = OneCycleLR,
    saving_div = 5,
    graph = True,
    sample_input = sample_tensor
)

AttributeError: 'NoneType' object has no attribute 'param_groups'